In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/scratch/cvlab/home/gwizdala/code/pidnet")

In [22]:
import numpy as np
import torch

from functools import reduce
from pathlib import Path

from powerlines.utils import load_yaml

folds = load_yaml(Path("configs/powerlines/folds.yaml"))
timestamps = set(reduce(lambda a, b: a + b, folds.values(), []))

labels_folder = Path("/scratch/cvlab/home/gwizdala/dataset/processed/daedalean/complete_frames/labels")
distance_masks_folder = Path(
    "/scratch/cvlab/home/gwizdala/dataset/processed/daedalean/complete_frames/distance_masks/visible"
)
poles_distance_masks_folder = Path(
    "/scratch/cvlab/home/gwizdala/dataset/processed/daedalean/complete_frames/poles_distance_masks"
)

cable_filepaths = list(filter(lambda path: int(path.stem) in timestamps, labels_folder.glob("*.png")))
cable_dist_mask_filepaths = list(filter(lambda path: int(path.stem) in timestamps, distance_masks_folder.glob("*.npy")))
poles_filepaths = list(filter(lambda path: int(path.stem) in timestamps, poles_distance_masks_folder.glob("*.npy")))
assert len(cable_filepaths) == len(poles_filepaths) == len(cable_dist_mask_filepaths)

In [23]:
from torch import nn
from tqdm import tqdm

from powerlines.data.utils import load_processed_label_image, downsample_labels

downsampler = nn.MaxPool2d(kernel_size=2, stride=2).cuda()

downsample_factor = 32
frame_size = int(3000 / downsample_factor) * int(4096 / downsample_factor)
total_pos_count = 0

In [24]:
max_pooling = nn.MaxPool2d(kernel_size=(downsample_factor, downsample_factor), stride=(downsample_factor, downsample_factor))

def min_pooling(distance_mask: torch.Tensor):
    # Assuming distance mask value is rescaled to [0, 1] range
    inverted_distance_mask = 1 - distance_mask
    pooled = max_pooling(inverted_distance_mask)
    return 1 - pooled

In [29]:
import numpy as np

# Cable frequency
total_cable_px = 0
for filepath in tqdm(cable_dist_mask_filepaths):
    # label = load_processed_label_image(filepath)
    distance_mask = np.clip(np.load(filepath), a_min=0, a_max=128) / 128
    
    # downsampled_label = downsample_labels(
    #     torch.from_numpy(label).unsqueeze(0).float(), grid_size=downsample_factor, adjust_to_divisible=False
    # ).long().detach().cpu().numpy()
    downsampled_dist_mask = min_pooling(torch.from_numpy(distance_mask).unsqueeze(0).float()).detach().cpu().numpy()

    # pos_count = np.count_nonzero(downsampled_label == 1)
    pos_count = np.count_nonzero(downsampled_dist_mask <= 0.25)
    total_pos_count += pos_count

pos_frequency = total_pos_count / (len(cable_filepaths) * frame_size)
neg_frequency = 1 - pos_frequency

print(f"Cables frequency: {pos_frequency}")
print(f"Cables weight: {1 / pos_frequency}")
print(f"Background weight: {1 / neg_frequency}")

100%|██████████| 865/865 [01:09<00:00, 12.52it/s]

Cables frequency: 0.04610059668096215
Cables weight: 21.691693209970172
Background weight: 1.048328572720098


In [30]:
from powerlines.data.utils import nevbw_labels_from_distance_mask

# Poles frequency
total_poles_px = 0
for filepath in tqdm(poles_filepaths):
    poles_distance_mask = np.clip(np.load(filepath), a_min=0, a_max=128) / 128
    # poles_label = nevbw_labels_from_distance_mask(poles_distance_mask)
    # downsampled_poles_label = downsample_labels(
    #     torch.from_numpy(poles_label).float(), grid_size=downsample_factor, adjust_to_divisible=False
    # ).long().detach().cpu().numpy()
    downsampled_poles_dist_mask = min_pooling(torch.from_numpy(poles_distance_mask).unsqueeze(0).float()).detach().cpu().numpy()

    # pos_count = np.count_nonzero(downsampled_poles_label == 1)
    pos_count = np.count_nonzero(downsampled_poles_dist_mask <= 0.25)
    total_pos_count += pos_count

pos_frequency = total_pos_count / (len(poles_filepaths) * frame_size)
neg_frequency = 1 - pos_frequency

print(f"Poles frequency: {pos_frequency}")
print(f"Poles weight: {1 / pos_frequency}")
print(f"Background weight: {1 / neg_frequency}")

100%|██████████| 865/865 [01:15<00:00, 11.51it/s]

Poles frequency: 0.06273103906395675
Poles weight: 15.941071834956551
Background weight: 1.0669296025777997


In [ ]:
# Weights are virtually identical for 16x and 32x downsampled labels, both for cables and poles LABELS
# In downsampled DISTANCE MASKS the 0-valued pixels are LESS frequent than in labels, both for cables and poles (see increased cables/poles weights)
#  by 0.75% for poles and 0.3% for cables (when converting to frequency)
# However, when thresholding distance masks with 0.25 the frequency of foreground more than doubles, both for for cables and poles